In [6]:
from pyseqrna import pyseqrna_utils as pu
from pyseqrna import quality_check as qc
from pyseqrna import quality_trimming as qt
from pyseqrna import  aligners as al
from pyseqrna import pyseqrna_stats as ps
from pyseqrna import quantification as quants
from pyseqrna import differential_expression as de
from pyseqrna import pyseqrna_plots as pp
from pyseqrna import multimapped_groups as mmg
import pandas as pd
import dill

In [7]:
from pyseqrna.pyseqrna_utils import PyseqrnaLogger

log = PyseqrnaLogger(mode='w', log='pp')

log.info("Analysis started")

[20:52:38]  <ipython-input-7-bf476bf393c3> :: INFO : Analysis started
[20:52:38]  <ipython-input-7-bf476bf393c3> :: INFO : Analysis started


In [8]:
data = pu.read_input_file("/home/naveen/Downloads/pyseqrna_test/input_sample.txt", "/home/naveen/Downloads/pyseqrna_test/" , paired=True)
samples= data['samples']

[20:52:41]  pyseqrna_utils :: INFO : Reading input samples File 
[20:52:41]  pyseqrna_utils :: INFO : Input file /home/naveen/Downloads/pyseqrna_test/input_sample.txt read succesfully
[20:52:41]  pyseqrna_utils :: INFO : Combination created succesfully from /home/naveen/Downloads/pyseqrna_test/input_sample.txt
[20:52:41]  pyseqrna_utils :: INFO : targets dataframe for differenatial created succesfully from /home/naveen/Downloads/pyseqrna_test/input_sample.txt


In [11]:
a, b = qc.fastqcRun(sampleDict=samples,paired=True, slurm=True,cpu=30,mem=200)

[20:53:18]  pyseqrna_utils :: INFO : Config generated succesfully from /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pyseqrna/param/fastqc.ini
[20:53:18]  quality_check :: INFO : Using default config file fastqc.ini
[20:53:18]  pyseqrna_utils :: INFO : Succesfully created directory /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/fastqc_results
[20:53:18]  quality_check :: INFO : Job submitted on slurm successfully for SRR16832244_1.fastq.gz and SRR16832244_2.fastq.gz with 36113
[20:53:18]  quality_check :: INFO : Job submitted on slurm successfully for SRR16832245_1.fastq.gz and SRR16832245_2.fastq.gz with 36114
[20:53:18]  quality_check :: INFO : Job submitted on slurm successfully for SRR16832246_1.fastq.gz and SRR16832246_2.fastq.gz with 36115
[20:53:18]  quality_check :: INFO : Job submitted on slurm successfully for SRR16832247_1.fastq.gz and SRR16832247_2.fastq.gz with 36116
[20:53:18]  quality_check :: INFO : Job submitted on slurm successfully for SRR1683

In [10]:
targets= data['targets']
comb= data['combinations']

In [9]:
pu.make_directory("pySeqRNA_results")

[20:52:46]  pyseqrna_utils :: INFO : Succesfully created directory /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results


'/home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results'

In [13]:

a = qt.trim_galoreRun(sampleDict=samples, paired=True, slurm=True, cpu=40,mem=200)

[21:24:11]  pyseqrna_utils :: INFO : Config generated succesfully from /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pyseqrna/param/trim_galore.ini
[21:24:11]  quality_trimming :: INFO : Using default config file trim_galore.ini
[21:24:11]  pyseqrna_utils :: INFO : Succesfully created directory /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/trim_galore_results
[21:24:11]  quality_trimming :: INFO : Job successfully submited for SRR16832244_1.fastq.gz and SRR16832244_2.fastq.gz with 36131
[21:24:11]  quality_trimming :: INFO : Job successfully submited for SRR16832245_1.fastq.gz and SRR16832245_2.fastq.gz with 36132
[21:24:11]  quality_trimming :: INFO : Job successfully submited for SRR16832246_1.fastq.gz and SRR16832246_2.fastq.gz with 36133
[21:24:11]  quality_trimming :: INFO : Job successfully submited for SRR16832247_1.fastq.gz and SRR16832247_2.fastq.gz with 36134
[21:24:11]  quality_trimming :: INFO : Job successfully submited for SRR16832248_1.fastq.gz a

In [ ]:
a[1]

In [15]:
aligner = al.STAR_Aligner(genome="/home/naveen/Downloads/pyseqrna_test/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa", slurm=True)

[21:44:37]  pyseqrna_utils :: INFO : Config generated succesfully from /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pyseqrna/param/STAR.ini
[21:44:37]  aligners :: INFO : Using default config file STAR.ini


In [16]:
j = aligner.build_index()

[21:44:41]  pyseqrna_utils :: INFO : Succesfully created directory /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_index
[21:44:41]  aligners :: INFO : Arabidopsis_thaliana.TAIR10.dna.toplevel.fa copied successfully in star_index
[21:44:41]  aligners :: INFO : Job successfully submited for /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_index/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa with 36141 for indexing


In [17]:
aligner.check_index()

True

In [19]:
outalign, job = aligner.run_Alignment(target=a[0], pairedEND=True, cpu=40, mem=200)

[21:52:23]  pyseqrna_utils :: INFO : Succesfully created directory /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results
[21:52:23]  aligners :: INFO : Job successfully submited for WT1_val_1.fq.gz and WT1_val_2.fq.gz with 36151 for alignment
[21:52:24]  aligners :: INFO : Job successfully submited for WT2_val_1.fq.gz and WT2_val_2.fq.gz with 36152 for alignment
[21:52:24]  aligners :: INFO : Job successfully submited for WT3_val_1.fq.gz and WT3_val_2.fq.gz with 36153 for alignment
[21:52:24]  aligners :: INFO : Job successfully submited for PF1_val_1.fq.gz and PF1_val_2.fq.gz with 36154 for alignment
[21:52:24]  aligners :: INFO : Job successfully submited for PF2_val_1.fq.gz and PF2_val_2.fq.gz with 36155 for alignment
[21:52:24]  aligners :: INFO : Job successfully submited for PF3_val_1.fq.gz and PF3_val_2.fq.gz with 36156 for alignment
[21:52:24]  aligners :: INFO : Job successfully submited for F1_val_1.fq.gz and F1_val_2.fq.gz with 36157 for alignment
[

In [23]:
ps.getUniquely_mapped("pySeqRNA_results/star_results/F1Aligned.out.bam")

36530341

In [24]:
df= ps.align_stats(sampleDict=samples,trimDict=a[0],bamDict=outalign, pairedEND=True)

In [25]:
df.to_excel("pySeqRNA_results/alignment_stats.xlsx", index=False)

In [26]:
df

,Sample,Input_reads2x,Cleaned2x,%_Cleaned2x,Aligned,%_Aligned,Uniquely_mapped,%_Uniquely_mapped,Multi_mapped,%_Multi_mapped
0,WT1,39641944,39625770,99.96,38958658,98.32,38338183,98.41,620475,1.59
1,WT2,36836982,36822032,99.96,36189715,98.28,35690142,98.62,499573,1.38
2,WT3,31803522,31792636,99.97,31258664,98.32,30864420,98.74,394244,1.26
3,PF1,33534220,33523266,99.97,32981811,98.38,32576574,98.77,405237,1.23
4,PF2,33522038,33510196,99.96,32893071,98.16,32474288,98.73,418783,1.27
5,PF3,31857100,31845818,99.96,31355902,98.46,30955952,98.72,399950,1.28
6,F1,37747846,37734534,99.96,37126363,98.39,36530341,98.39,596022,1.61
7,F2,35905062,35891460,99.96,35290295,98.33,34721820,98.39,568475,1.61
8,F3,33266248,33254296,99.96,32775791,98.56,32300078,98.55,475713,1.45


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import seaborn as sns

In [ ]:
sample = list(df['Sample'])
creads = list(df['Input_reads2x'])
iread = list(df['%_Cleaned2x'])
tc= list(100-df['%_Cleaned2x'])

plt.barh(sample,iread,0.6,label="cleaned Reads", color='skyblue')
plt.barh(sample, tc, 0.6,left=iread, label="discarded", color='tomato')
plt.margins(y=0.01)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("% Number of reads", fontsize=18)
plt.ylabel("Samples", fontsize=18)
plt.legend(['Cleaned', 'Discarded'], bbox_to_anchor=(1.0, 1.0), loc='upper left', fontsize=15)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)

# plt.legend(frameon=False, fontsize=17)



In [ ]:
tc

In [31]:
a= quants.featureCount(bamDict=outalign, gff="/home/naveen/Downloads/pyseqrna_test/Arabidopsis_thaliana.TAIR10.51.gff3", outDir="pySeqRNA_results")

[00:37:49]  pyseqrna_utils :: INFO : Config generated succesfully from /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pyseqrna/param/featureCount.ini
[00:37:49]  quantification :: INFO : Using default config file featureCount.ini


/home/naveen/SOFTWARE/miniconda3/envs/pySeqRNA/bin/featureCounts  -a /home/naveen/Downloads/pyseqrna_test/Arabidopsis_thaliana.TAIR10.51.gff3 -o pySeqRNA_results/Counts.txt -T 5 -g ID -t gene /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/WT1Aligned.out.bam /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/WT2Aligned.out.bam /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/WT3Aligned.out.bam /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/PF1Aligned.out.bam /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/PF2Aligned.out.bam /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/PF3Aligned.out.bam /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/F1Aligned.out.bam /home/naveen/Documents/Phd_work/ngs_data/pyseqrna/pySeqRNA_results/star_results/F2Aligned.out.bam /home/naveen/Docum

[00:39:58]  quantification :: INFO : Job successfully completed for feature count


In [ ]:
counts= pd.read_csv("pySeqRNA_results/Counts_final.txt", sep="\t")

In [ ]:
counts

In [ ]:
targets= data['targets']
comb= data['combinations']

In [ ]:
dd= de.runDESeq2(countDF=counts,targetFile=targets,design='sample', combination=comb)

In [ ]:
import os 
dd.to_excel(os.path.join("pySeqRNA_results","Raw_DEGs_all.xlsx"), index=False)

In [ ]:
dd= pd.read_excel("pySeqRNA_results/Raw_DEGs_all.xlsx")
filtered_DEG = de.degFilter(degDF=dd, CompareList=comb, FDR=0.05, FOLD=1)

In [ ]:
summary= filtered_DEG['summary']

In [ ]:
wd= pd.ExcelWriter(os.path.join("pySeqRNA_results","filtered_DEGs.xlsx"))
for key, value in filtered_DEG['filtered'].items():
    value.to_excel(wd,sheet_name=key)
    wd.save()
wd.close()

In [ ]:
pu.getGenes(os.path.join("pySeqRNA_results","filtered_DEGs.xlsx"), combinations=comb)

In [ ]:
from pyseqrna import normalize_counts as nc

In [ ]:
rpkm = nc.Normalization(countFile="pySeqRNA_results/Counts_final.txt", featureFile="pyseqrna/example/data/arabidopsis10.gff")

In [ ]:
rpk = rpkm.meanRatioCount()

In [ ]:
rpk.to_excel("pySeqRNA_results/rpkm.xlsx")

In [ ]:
rcount= pd.read_excel("pySeqRNA_results/rpkm.xlsx")

In [ ]:
rcount

In [ ]:
from pyseqrna import pyseqrna_plots as pp

In [ ]:
heatmap, ax = pp.plotHeatmap(dd,combinations=comb,num=50, type='deg')

In [ ]:
from pyseqrna import gene_ontology as go

In [ ]:
bdata= go.query('athaliana')

In [ ]:
gdata = go.preprocessBioMart(bdata)

In [ ]:
results = go.enrichGO('athaliana',"pySeqRNA_results/diff_genes/C1-D6.txt")

In [ ]:
results

In [ ]:
df=bdata
import numpy as np
df2 = df[df['GO_ID'].notna()]
gg = list(df2['Gene'])
x = np.array(gg)

bg_count = len(np.unique(x))

bg_count


In [ ]:
cd= pd.read_excel("pySeqRNA_results.1/M1-V6_deg.xlsx")

In [ ]:
gdata

In [ ]:
from pyseqrna import pathway


In [ ]:
dk = pathway.enrichKEGG(file="pySeqRNA_results/diff_genes/C1-D6.txt", species='ath')